# Text-To-Speech (TTS) 
(also called speech synthesis)

# Text-to-speech datasets
(only texts)

**Some datasets for ASR can be used to train TTS model.**

## LJSpeech

single speaker reading sentences from 7 non-fiction books in English.

## Multilingual LibriSpeech

multilingual extension of the LibriSpeech dataset for developing multilingual TTS systems and exploring cross-lingual speech synthesis techniques.

## VCTK (Voice Cloning Toolkit)

audio recordings of 110 English speakers with various accents for training TTS models with varied voices and accents, enabling more natural and diverse speech synthesis.

## Libri-TTS/ LibriTTS-R

multi-speaker English corpus of approximately 585 hours of read English speech at 24kHz sampling rate.

# Pre-trained models for text-to-speech

## SpeechT5 model

In [1]:
from transformers import SpeechT5Processor, SpeechT5ForTextToSpeech

processor = SpeechT5Processor.from_pretrained("microsoft/speecht5_tts")
model = SpeechT5ForTextToSpeech.from_pretrained("microsoft/speecht5_tts")

/home/fds/miniconda3/envs/py310_pytorch/lib/python3.10/site-packages/transformers/utils/hub.py:124: FutureWarning: Using `TRANSFORMERS_CACHE` is deprecated and will be removed in v5 of Transformers. Use `HF_HOME` instead.
  warnings.warn(


In [2]:
inputs = processor(text="Don't count the days, make the days count.", return_tensors="pt")

In [3]:
from datasets import load_dataset

embeddings_dataset = load_dataset("Matthijs/cmu-arctic-xvectors", split="validation")

import torch

speaker_embeddings = torch.tensor(embeddings_dataset[7306]["xvector"]).unsqueeze(0)

Generating validation split:   0%|          | 0/7931 [00:00<?, ? examples/s]

In [7]:
spectrogram = model.generate_speech(inputs["input_ids"], speaker_embeddings)